# 구내식당 식수 인원 예측 AI 경진대회 (dacon)
- 대회 overview: https://dacon.io/competitions/official/235743/overview/description
- 평가기준 : MAE

## 1st try
- 이번 풀잎스쿨에서 알게된 fastai 라이브러리를 적용하여 인원 예측을 해보기로 하였다.
- 예측은 RandomForestRegressor를 사용하여 진행
- 1st try에서는 아무런 eda나 feature engineering 없이 진행하여 그 결과를 확인해본다.

In [1]:
# 필요한 패키지들을 가져온다.
import pandas as pd
import numpy as np
import fastai
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
import gc
import math
from sklearn.metrics import mean_absolute_error as mae
from fastai.imports import *
from fastai.tabular import *

In [2]:
'''
주피터 노트북을 사용하거나 순수 파이썬을 사용하거나 사용할 라이브러리는 불러와져야 합니다.
주피터 노트북 상단에서 "autoreload"명령어를 사용하면 모듈의 소스 코드 수정이 발생할 때, 자동으로 새로운 모듈을 사용하도록 업데이트 됩니다.
원하는 그래프를 노트북에서 보려면 "matplotlib inline"을 사용하세요.
'''

%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
# fastai version 확인
fastai.__version__

'2.4'

In [4]:
# train/test/sample submission 데이터를 가져온다.
path = '../cafeteria_prediction/data/'
df = pd.read_csv(f'{path}train.csv')
test = pd.read_csv(f'{path}test.csv')
submission = pd.read_csv(f'{path}sample_submission.csv')
df.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,"모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐어채무침 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",1039.0,331.0
1,2016-02-02,화,2601,50,173,319,0.0,"모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌개 시래기조림 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무침 요구르트 배추겉절이 (배추,고추가루:국내산)","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무침 바나나 포기김치 (배추,고추가루:국내산)",867.0,560.0
2,2016-02-03,수,2601,56,180,111,0.0,"모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국 느타리호박볶음 포기김치 (배추,고추가루:국내산)","카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 견과류조림 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기전 (돼지고기:국내산) 새송이버섯볶음 포기김치 (배추,고추가루:국내산)",1017.0,573.0


In [5]:
df.columns

Index(['일자', '요일', '본사정원수', '본사휴가자수', '본사출장자수', '본사시간외근무명령서승인건수',
       '현본사소속재택근무자수', '조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계'],
      dtype='object')

In [6]:
# fastai old version(0.7.0)에 있는 함수들을 사용하기 위해 따로 함수 지정해준다.
# proc_df : 변수들을 수치화해준다. 결측치는 median으로 채워준다.

def numericalize(df, col, name, max_n_cat):
    """ Changes the column col from a categorical type to it's integer codes.
    """
    if not is_numeric_dtype(col) and ( max_n_cat is None or len(col.cat.categories)>max_n_cat):
        df[name] = pd.Categorical(col).codes+1

def fix_missing(df, col, name, na_dict):
    """ Fill missing data in a column of df with the median, and add a {name}_na column
    which specifies if the data was missing.
    """
    if is_numeric_dtype(col):
        if pd.isnull(col).sum() or (name in na_dict):
            df[name+'_na'] = pd.isnull(col)
            filler = na_dict[name] if name in na_dict else col.median()
            df[name] = col.fillna(filler)
            na_dict[name] = filler
    return na_dict

def proc_df(df, y_fld=None, skip_flds=None, ignore_flds=None, do_scale=False, na_dict=None,
            preproc_fn=None, max_n_cat=None, subset=None, mapper=None):
    """ proc_df takes a data frame df and splits off the response variable, and
    changes the df into an entirely numeric dataframe.

    Parameters:
    -----------
    df: The data frame you wish to process.

    y_fld: The name of the response variable

    skip_flds: A list of fields that dropped from df.

    ignore_flds: A list of fields that are ignored during processing.

    do_scale: Standardizes each column in df. Takes Boolean Values(True,False)

    na_dict: a dictionary of na columns to add. Na columns are also added if there
        are any missing values.

    preproc_fn: A function that gets applied to df.

    max_n_cat: The maximum number of categories to break into dummy values, instead
        of integer codes.

    subset: Takes a random subset of size subset from df.

    mapper: If do_scale is set as True, the mapper variable
        calculates the values used for scaling of variables during training time (mean and standard deviation).

    Returns:
    --------
    [x, y, nas, mapper(optional)]:

        x: x is the transformed version of df. x will not have the response variable
            and is entirely numeric.

        y: y is the response variable

        nas: returns a dictionary of which nas it created, and the associated median.

        mapper: A DataFrameMapper which stores the mean and standard deviation of the corresponding continuous
        variables which is then used for scaling of during test-time.

    Examples:
    ---------
    >>> df = pd.DataFrame({'col1' : [1, 2, 3], 'col2' : ['a', 'b', 'a']})
    >>> df
       col1 col2
    0     1    a
    1     2    b
    2     3    a

    note the type of col2 is string

    >>> train_cats(df)
    >>> df

       col1 col2
    0     1    a
    1     2    b
    2     3    a

    now the type of col2 is category { a : 1, b : 2}

    >>> x, y, nas = proc_df(df, 'col1')
    >>> x

       col2
    0     1
    1     2
    2     1

    >>> data = DataFrame(pet=["cat", "dog", "dog", "fish", "cat", "dog", "cat", "fish"],
                 children=[4., 6, 3, 3, 2, 3, 5, 4],
                 salary=[90, 24, 44, 27, 32, 59, 36, 27])

    >>> mapper = DataFrameMapper([(:pet, LabelBinarizer()),
                          ([:children], StandardScaler())])

    >>>round(fit_transform!(mapper, copy(data)), 2)

    8x4 Array{Float64,2}:
    1.0  0.0  0.0   0.21
    0.0  1.0  0.0   1.88
    0.0  1.0  0.0  -0.63
    0.0  0.0  1.0  -0.63
    1.0  0.0  0.0  -1.46
    0.0  1.0  0.0  -0.63
    1.0  0.0  0.0   1.04
    0.0  0.0  1.0   0.21
    """
    if not ignore_flds: ignore_flds=[]
    if not skip_flds: skip_flds=[]
    if subset: df = get_sample(df,subset)
    ignored_flds = df.loc[:, ignore_flds]
    df.drop(ignore_flds, axis=1, inplace=True)
    df = df.copy()
    if preproc_fn: preproc_fn(df)
    if y_fld is None: y = None
    else:
        if not is_numeric_dtype(df[y_fld]): df[y_fld] = df[y_fld].cat.codes
        y = df[y_fld].values
        skip_flds += [y_fld]
    df.drop(skip_flds, axis=1, inplace=True)

    if na_dict is None: na_dict = {}
    for n,c in df.items(): na_dict = fix_missing(df, c, n, na_dict)
    if do_scale: mapper = scale_vars(df, mapper)
    for n,c in df.items(): numericalize(df, c, n, max_n_cat)
    df = pd.get_dummies(df, dummy_na=True)
    df = pd.concat([ignored_flds, df], axis=1)
    res = [df, y, na_dict]
    if do_scale: res = res + [mapper]
    return res

In [7]:
df.head(1)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
0,2016-02-01,월,2601,50,150,238,0.0,"모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐어채무침 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 청포묵무침 요구르트 포기김치 (배추,고추가루:국내산)","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 포기김치 (김치:국내산)",1039.0,331.0


In [8]:
print(df.shape)
print(df['일자'].nunique())  # 일자 중복 없음 확인
print(df.isna().sum()) # 결측치 없는 데이터인 것 확인

(1205, 12)
1205
일자                0
요일                0
본사정원수             0
본사휴가자수            0
본사출장자수            0
본사시간외근무명령서승인건수    0
현본사소속재택근무자수       0
조식메뉴              0
중식메뉴              0
석식메뉴              0
중식계               0
석식계               0
dtype: int64


In [10]:
# 자기개발 등등 석식계 0 인 날 처리
df.loc[(df['석식계'] < 1)&(df['석식메뉴'].str.contains('\*[^ㄱ-힣]')),'석식메뉴'] = '*'
df.loc[(df['석식메뉴'].str.contains('자기개발|자기계발|자기 개발|자기 계발')),'석식메뉴'] = '자기계발의날'
df.loc[(df['석식메뉴'].str.contains('가정의')),'석식메뉴'] = '가정의날'

In [11]:
# 중식용이랑 석식용으로 데이터 만들기
df1_trn, y1_trn, nas_1 = proc_df(df.drop(columns = ['석식계','석식메뉴']), '중식계')
df2_trn, y2_trn, nas_2 = proc_df(df.drop(columns = ['중식메뉴','중식계']), '석식계') # test 데이터에서도 중식계 feature를 알 수 없으므로 맞춰준다.

In [12]:
# proc_df 결과확인
display(df1_trn.head(2))
display(df2_trn.head(2))

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴
0,1,4,2601,50,150,238,0.0,764,256
1,2,5,2601,50,173,319,0.0,161,237


,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,석식메뉴
0,1,4,2601,50,150,238,0.0,764,301
1,2,5,2601,50,173,319,0.0,161,1032


In [13]:
df.loc[df['중식계'] < 10, ['중식메뉴']]

,중식메뉴


In [14]:
print(y1_trn[:3])
print(y2_trn[:3])

[1039.  867. 1017.]
[331. 560. 573.]


In [15]:
print(nas_1)
print(nas_2)
# 결측치는 없는 것으로...

{}
{}


In [16]:
# train valid split할 지점 확인 (7대 3으로 나눌 것임)
print(df1_trn.shape)
print(df1_trn.shape[0]*.7)
print(df1_trn.shape[0]*.3)

print(df2_trn.shape)
print(df2_trn.shape[0]*.7)
print(df2_trn.shape[0]*.3)

(1205, 9)
843.5
361.5
(1205, 9)
843.5
361.5


In [17]:
# 843 / 362
def split_vals(a,n): return a[:n], a[n:]   # 시간순으로 split

n_valid = 362
n_trn = len(df1_trn)-n_valid
X1_train, X1_valid = split_vals(df1_trn, n_trn)
y1_train, y1_valid = split_vals(y1_trn, n_trn)
raw1_train, raw1_valid = split_vals(df, n_trn)

# n_trn은 동일하니 생략
X2_train, X2_valid = split_vals(df2_trn, n_trn)
y2_train, y2_valid = split_vals(y2_trn, n_trn)
raw2_train, raw2_valid = split_vals(df, n_trn)

In [18]:
dinner_non_list = df.loc[df['석식계'] == 0].index.values
X2_train[X2_train.index.isin(dinner_non_list)]

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,석식메뉴
204,205,3,2689,68,207,0,0.0,933,8
224,225,3,2705,166,225,0,0.0,72,8
244,245,3,2697,79,203,0,0.0,999,8
262,263,3,2632,75,252,0,0.0,397,8
281,282,3,2627,53,235,0,0.0,768,8
306,307,3,2626,45,304,0,0.0,713,8
327,328,3,2637,43,265,0,0.0,346,932
346,347,3,2648,58,259,0,0.0,144,8
366,367,3,2839,254,246,0,0.0,140,9
392,393,1,2642,177,303,45,0.0,150,8


In [19]:
test['석식메뉴'].unique().tolist()
# test데이터에는 석식메뉴 *이 없으므로 따로 처리 필요 X

['흑미밥 얼큰순두부찌개 쇠고기우엉볶음 버섯햄볶음 (New)아삭이고추무절임 포기김치 ',
 '충무김밥 우동국물 오징어무침 꽃맛살샐러드 얼갈이쌈장무침 석박지 ',
 '흑미밥 물만둣국 카레찜닭 숯불양념꼬지어묵 꼬시래기무침 포기김치 ',
 '흑미밥 동태탕 돈육꽈리고추장조림 당면채소무침 모자반무침 포기김치 ',
 '흑미밥 바지락살국 쇠고기청경채볶음 두부구이*볶은김치 머위된장무침 백김치 ',
 '오므라이스 가쓰오장국 빌소세지구이*구운채소 단감치커리무침 양념고추지 겉절이김치 ',
 '흑미밥 계란파국 돈육두루치기 감자채파프리카볶음 세발나물오리엔탈무침 포기김치 ',
 '유부초밥/추가밥 온메밀소바 국물떡볶이 순대찜*소금 청경채겉절이 포기김치 ',
 '흑미밥 냉이국 반반치킨 꼬막채소무침 청경채찜 포기김치 ',
 '흑미밥 미역국 매운소불고기 단호박두부탕수 메추리알장조림 석박지 ',
 '흑미밥 참치김치찌개 오징어굴소스볶음 차돌비빔국수 건새우무나물 포기김치 ',
 '흑미밥 순두부백탕 수제치킨까스 쫄면채소무침 얼갈이나물 포기김치 ',
 '흑미밥 손수제비국 쇠고기낙지볶음 카레홍합찜 쑥갓나물 포기김치 ',
 '곤드레밥 황태국 찰떡떡갈비조림 계란후라이 재래김*달래양념장 무생채 ',
 '흑미밥 바지락된장찌개 제육볶음 양배추숙*쌈장 노가리고추조림 겉절이김치 ',
 '흑미밥 버섯들깨탕 아귀콩나물찜 콤비네이션피자 돌나물&된장소스 포기김치 ',
 '흑미밥 동태알탕 깐풍육 고사리볶음 오이무침 포기김치 ',
 '흑미밥 쇠고기무국 춘전닭갈비 뉴욕핫도그 유채나물된장무침 포기김치 ',
 '애플카레라이스 팽이장국 가지탕수 소떡소떡 오복지무침 포기김치 ',
 '흑미밥 계란파국 쭈꾸미불고기 모둠채소전*장 씨앗콩자반 포기김치 ',
 '흑미밥 스팸김치찌개 삼치구이*와사비장 브로콜리깨소스무침 연근조림 포기김치 ',
 '흑미밥 냉이김칫국 해물우동볶음 날치알계란찜 솎음열무나물 포기김치 ',
 '흑미밥 (New)수제오떡탕 매운족발볶음 크래미오이보트샐러드 청경채나물 겉절이김치 ',
 '흑미밥 짬뽕국 쇠고기탕수 고추잡채*꽃빵 

In [20]:
# model fitting 후 score확인 함수
def print_score(m, X_train, y_train, X_valid, y_valid):
    res = [mae(m.predict(X_train), y_train), mae(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [17]:
#??RandomForestRegressor

In [21]:
# del m1
# gc.collect()
# oob score : out of bag (무작위 샘플링 후 샘플링에 사용되지 않은 데이터들을 또 하나의 테스트 및 평가 대상으로 잡고 score를 내는 것)
m1 = RandomForestRegressor(n_estimators=40, criterion = 'mae', min_samples_leaf=3, n_jobs=-1, oob_score=True) # fitting결과를 보고 parameter 조정가능
%time m1.fit(X1_train, y1_train)
print_score(m1, X1_train, y1_train, X1_valid, y1_valid)

# 결과 :  [train prediction mae, valid prediction mae, train predict score, valid predict score, oob score]

Wall time: 386 ms
[48.36467971530249, 91.37151243093922, 0.8941184264549046, 0.7227710472366633, 0.7620099021290054]


In [22]:
# del m2
# gc.collect()
m2 = RandomForestRegressor(n_estimators=40, criterion = 'mae', min_samples_leaf=3, n_jobs=-1, oob_score=True)
%time m2.fit(X2_train, y2_train)
print_score(m2, X2_train, y2_train, X2_valid, y2_valid)

Wall time: 437 ms
[31.133199881376036, 80.35366022099448, 0.8546123011375776, 0.46761736367830276, 0.7057438244352116]


In [23]:
# test data prediction하기
test = pd.read_csv(f'{path}test.csv')
test['중식계'] = np.nan
test_X1, test_y1, nas_1 = proc_df(test, '중식계')

test_X1.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴
0,1,3,2983,88,182,5,358.0,25,26,46
1,2,2,2983,104,212,409,348.0,11,9,13
2,3,1,2983,270,249,0,294.0,46,45,29


In [24]:
test['석식계'] = np.nan
test_X2, test_y2, nas_2 = proc_df(test.drop(columns = ['중식메뉴','중식계']), '석식계')
test_X2.head(3)

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,석식메뉴
0,1,3,2983,88,182,5,358.0,25,46
1,2,2,2983,104,212,409,348.0,11,13
2,3,1,2983,270,249,0,294.0,46,29


In [26]:
test_X1 = test_X1.drop(columns = '석식메뉴')
pred1 = m1.predict(test_X1)

In [27]:
pred2 = m2.predict(test_X2)

submission['중식계'] = pred1
submission['석식계'] = pred2

submission

,일자,중식계,석식계
0,2021-01-27,985.2500,475.0875
1,2021-01-28,985.7750,558.3625
2,2021-01-29,740.2625,464.4500
3,2021-02-01,1250.6250,591.6125
4,2021-02-02,1026.7000,584.0500
5,2021-02-03,991.1500,0.0000
6,2021-02-04,1080.1625,565.5375
7,2021-02-05,754.0375,109.8250
8,2021-02-08,1239.3125,594.1000
9,2021-02-09,1034.9250,574.2000


In [28]:
submission['중식계'] = submission['중식계'].apply(lambda x: round(x))
submission['석식계'] = submission['석식계'].apply(lambda x: round(x))

In [29]:
submission.to_csv(f'{path}submission_ver4.csv', index=False)
submission

,일자,중식계,석식계
0,2021-01-27,985,475
1,2021-01-28,986,558
2,2021-01-29,740,464
3,2021-02-01,1251,592
4,2021-02-02,1027,584
5,2021-02-03,991,0
6,2021-02-04,1080,566
7,2021-02-05,754,110
8,2021-02-08,1239,594
9,2021-02-09,1035,574


In [ ]:
'''
-결과-
완전 하위권
석식계 예측값이 0이나 너무 낮은 숫자로 나오는 경우가 있다. 이 경우들에 관해 살펴봐야함 
* fastai 라이브러리에서 중식메뉴, 석식메뉴를 수치화할때 train과 test의 수치화가 동일한 것인지? 확인필요

- 더 해봐야 할 것들 -
1. 날씨 정보 : 비가 오는 날, 너무 더운 날, 너무 추운 날에 구내식당 이용률이 높을 수 있다.
2. 메뉴 embedding : 메뉴가 구내식당 이용 여부에 영향을 줄 수 있다.
'''